This notebook contains exploratory data analysis, feature extraction and writes the table to a Postgresql database. 
If you run it for the first time: 
1. Make sure you have created an .env where you put your database credentials, for a full list check utils/db.py. 
2. Change 'SHOULD_CREATE_DB_TABLE' to True after you completed 1.

### Imports

In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
import os
import pandas as pd
import numpy as np
from utils.db import engine
import calendar

In [48]:
SHOULD_CREATE_DB_TABLE = False # if set to True, it creates the table. You need a Postgresql connection and creds

### Load Dataframe from repo and exploring it. 

In [49]:
df = pd.read_csv("../datasets/marketing_data.csv")

df["Date"] = pd.to_datetime(df["Date"])
df = df.drop_duplicates() # there were no duplicates but good to be kept

df.head(3)


,Year,Quarter,Month,Week,Date,Country,Media Category,Media Name,Communication,Campaign Category,Product,Campaign Name,Revenue,Cost
0,2020,2020 Q3,2020M08,35,2020-08-24,DK,online,Interscroller,Tactical,Category 3,Product 1,Campaign 10,411.522496,9778.544045
1,2020,2020 Q3,2020M08,35,2020-08-24,DK,online,Interscroller,Tactical,Category 3,Product 1,Campaign 11,228.913948,5083.770000
2,2020,2020 Q3,2020M08,35,2020-08-24,DK,online,Interscroller,Branding,Category 5,Product 1,Campaign 12,71.051079,294.960000


In [50]:
# relatively small dataset, important for considering the choice for the LLM and it's size
print("Shape of the df is :", df.shape)


Shape of the df is : (12012, 14)


In [51]:
print("Columns in df :", df.columns)

Columns in df : Index(['Year', 'Quarter', 'Month', 'Week', 'Date', 'Country', 'Media Category',
       'Media Name', 'Communication', 'Campaign Category', 'Product',
       'Campaign Name', 'Revenue', 'Cost'],
      dtype='str')


In [52]:
# no null values
df.isna().sum()

Year                 0
Quarter              0
Month                0
Week                 0
Date                 0
Country              0
Media Category       0
Media Name           0
Communication        0
Campaign Category    0
Product              0
Campaign Name        0
Revenue              0
Cost                 0
dtype: int64

In [53]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 12012 entries, 0 to 12011
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Year               12012 non-null  int64         
 1   Quarter            12012 non-null  str           
 2   Month              12012 non-null  str           
 3   Week               12012 non-null  int64         
 4   Date               12012 non-null  datetime64[us]
 5   Country            12012 non-null  str           
 6   Media Category     12012 non-null  str           
 7   Media Name         12012 non-null  str           
 8   Communication      12012 non-null  str           
 9   Campaign Category  12012 non-null  str           
 10  Product            12012 non-null  str           
 11  Campaign Name      12012 non-null  str           
 12  Revenue            12012 non-null  float64       
 13  Cost               12012 non-null  float64       
dtypes: datetime64[us]

In [54]:
df.describe()

,Year,Week,Date,Revenue,Cost
count,12012.000000,12012.000000,12012,12012.000000,12012.000000
mean,2021.797869,25.672661,2022-04-11 16:46:09.230769,6097.097361,8620.176900
min,2020.000000,1.000000,2020-08-24 00:00:00,0.005004,0.000000
25%,2021.000000,12.000000,2021-07-26 00:00:00,6.623539,0.000000
50%,2022.000000,25.000000,2022-04-18 00:00:00,315.093438,0.000000
75%,2023.000000,39.000000,2023-01-02 00:00:00,3090.130713,2051.198431
max,2023.000000,53.000000,2023-08-21 00:00:00,499504.849292,685826.662584
std,0.886312,15.262821,NaN,24170.560359,38746.307417


#### Preprocessing
The idea here is to reduce the chances for the LLM to hallucinate. LLMs love structure and precomputed metrics reduce the number of points of failure.

In [55]:
# Renaming columns
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)

# Precomputed metrics to help the LLM
df["profit"] = df["revenue"] - df["cost"]

df["roi"] = np.where(
    df["cost"] != 0,
    df["profit"] / df["cost"],
    np.nan
)

df["margin"] = np.where(
    df["revenue"] != 0,
    df["profit"] / df["revenue"],
    np.nan
)


# Parsed dates to make it easier for the LLM to filter and look in the table
df["quarter_number"] = df["date"].dt.quarter
df["month_number"] = df["date"].dt.month
df["month_name"] = df["month_number"].apply(lambda x: calendar.month_name[x])

# Ensuring the type is in desired format
df["date"] = pd.to_datetime(df["date"])
df["year"] = df["year"].astype(int)
df["week"] = df["week"].astype(int)
df["revenue"] = df["revenue"].astype(float)
df["cost"] = df["cost"].astype(float)
df["quarter_number"] = df["quarter_number"].astype(int)
df["month_number"] = df["month_number"].astype(int)

df.head(3)

,year,quarter,month,week,date,country,media_category,media_name,communication,campaign_category,product,campaign_name,revenue,cost,profit,roi,margin,quarter_number,month_number,month_name
0,2020,2020 Q3,2020M08,35,2020-08-24,DK,online,Interscroller,Tactical,Category 3,Product 1,Campaign 10,411.522496,9778.544045,-9367.021548,-0.957916,-22.761870,3,8,August
1,2020,2020 Q3,2020M08,35,2020-08-24,DK,online,Interscroller,Tactical,Category 3,Product 1,Campaign 11,228.913948,5083.770000,-4854.856052,-0.954972,-21.208214,3,8,August
2,2020,2020 Q3,2020M08,35,2020-08-24,DK,online,Interscroller,Branding,Category 5,Product 1,Campaign 12,71.051079,294.960000,-223.908921,-0.759116,-3.151380,3,8,August


### Writing the table for the first time. 

In [56]:
if SHOULD_CREATE_DB_TABLE: # this is now set to False, set it to True if you run the script for the first time, see 3rd code cell
    df.to_sql(
        "marketing_data",
        engine,
        if_exists="replace",
        index=False
    )
df = pd.read_sql("SELECT * FROM marketing_data", engine)
    
print(df.shape)

(12012, 20)


In [57]:
# displaying schema information in order to give the LLM exact details about it

query = """
SELECT 
    column_name,
    data_type,
    character_maximum_length
FROM information_schema.columns
WHERE table_name = 'marketing_data';
"""

schema_df = pd.read_sql(query, engine)
schema_df

,column_name,data_type,character_maximum_length
0,date,timestamp without time zone,None
1,week,bigint,None
2,revenue,double precision,None
3,cost,double precision,None
4,profit,double precision,None
5,roi,double precision,None
6,margin,double precision,None
7,quarter_number,bigint,None
8,month_number,bigint,None
9,year,bigint,None
